# LDA Topic Modelling
Code from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#9createbigramandtrigrammodels

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Notwendige Bibliotheken installieren und laden

In [2]:
!pip install gensim pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.


In [2]:
!python -m spacy download de

2023-06-18 16:48:19.492645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 16:48:20.787913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 86.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

## Rohdaten einlsen

In [4]:
df = pd.read_parquet('/content/drive/MyDrive/Masterthesis/2023-06-16_gesamtdaten_bis_2022.parquet')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Bearbeitung der Daten

In [13]:
# Convert to list
df_filtered = df[df['kommune'].isin(['Frankenthal', 'Lampertheim', 'Schwetzingen', 'Kreis Bergstraße'])]
data = df_filtered['text'].values.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import nltk
nltk.download('stopwords')
stopwords = stopwords.words('german')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['wieder', 'bedankt', 'sich', 'bei', 'den', 'ausgeschiedenen', 'ratsmitgliedern', 'siehe', 'beiliegender', 'redeentwurf', 'und', 'überreicht', 'ihnen', 'als', 'zeichen', 'der', 'anerkennung', 'und', 'des', 'dankes', 'einen', 'bildband', 'kernland', 'rheinland', 'pfalz']]


In [15]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # Initialize spacy 'de' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download de
    nlp = spacy.load('de_core_news_sm', disable=['parser', 'ner'])
    texts_out = []


    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['wieder', 'bedankt', 'sich', 'bei', 'den', 'ausgeschiedenen', 'ratsmitgliedern', 'siehe', 'beiliegender', 'redeentwurf', 'und', 'überreicht', 'ihnen', 'als', 'zeichen', 'der', 'anerkennung', 'und', 'des', 'dankes', 'einen', 'bildband', 'kernland', 'rheinland_pfalz']


In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams and Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams_trigrams = make_trigrams(data_words_bigrams)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams_trigrams)

print(data_lemmatized[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['bedanken', 'ausgeschieden', 'Ratsmitglieder', 'sehen', 'beiliegend', 'Redeentwurf', 'überreicht', 'Zeichen', 'Anerkennung', 'Bildband']]


Error:
* NER braucht sehr viel RAM und nur ein Teil der Daten können verabreitet werden

In [19]:
# TF-IDF
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_lemmatized)

corpus = [id2word.doc2bow(text) for text in data_lemmatized]
print(corpus[0][:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    #reassign
    corpus[i] = new_bow

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [20]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('Anerkennung', 1),
  ('Bildband', 1),
  ('Ratsmitglieder', 1),
  ('Redeentwurf', 1),
  ('Zeichen', 1),
  ('ausgeschieden', 1),
  ('bedanken', 1),
  ('beiliegend', 1),
  ('sehen', 1),
  ('überreicht', 1)]]

## LDA Modelle erstellen

### TF-IDF

#### 500 Themen

In [21]:
# Build LDA model
lda_model_tfidf_500 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=500,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#saving model to disk.
from gensim.test.utils import datapath

temp_file = datapath("/content/drive/MyDrive/Masterthesis/lda_model_tfidf_500")

lda_model_tfidf_500.save(temp_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 800 Themen

In [ ]:
# Build LDA model
lda_model_tfidf_800 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=800,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#saving model to disk.

temp_file = datapath("/content/drive/MyDrive/Masterthesis/lda_model_tfidf_800")

lda_model_tfidf_800.save(temp_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Modelle mit TDF 

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


#### 500 Themen

In [ ]:
# Build LDA model
lda_model_tdf_500 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=500,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#saving model to disk.

temp_file = datapath("/content/drive/MyDrive/Masterthesis/lda_model_tdf_500")

lda_model_tdf_500.save(temp_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### 800 Themen

In [ ]:
# Build LDA model
lda_model_tdf_800 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=800,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=2,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#saving model to disk.

temp_file = datapath("/content/drive/MyDrive/Masterthesis/lda_model_tdf_800")

lda_model_tdf_800.save(temp_file)

## Ergebnisse

In [2]:
df = pd.read_parquet("data/raw/2023-06-23_gesamtdaten_rhein_neckar.parquet")
df_filtered = df[df['kommune'].isin(['Frankenthal', 'Lampertheim', 'Schwetzingen', 'Kreis Bergstraße'])]
data = df_filtered['text'].values

### TF-IDF

#### 500 Themen

In [3]:
#loading model from disk
from gensim import  models

lda_tfidf_500 = models.ldamodel.LdaModel.load("data/lda_models/lda_model_tfidf_500")
id2word_tfidf_500 = corpora.Dictionary.load('data/lda_models/lda_model_tfidf_500.id2word')
corpus_tfidf_500 = corpora.MmCorpus("data/lda_models/tdidf_corpus")

In [4]:
pyLDAvis.enable_notebook()
vis_tfidf_500 = pyLDAvis.gensim.prepare(lda_tfidf_500, corpus_tfidf_500, id2word_tfidf_500, mds='mmds')
vis_tfidf_500

C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.561994 -0.304651       1        1  4.366065
205    0.180509 -0.603851       2        1  2.837298
299    0.195973 -0.443455       3        1  2.694570
85     0.276840 -0.557784       4        1  2.469734
11    -0.374727 -0.502410       5        1  2.365426
...         ...       ...     ...      ...       ...
198   -0.033419  0.047887     496        1  0.001126
197   -0.033419  0.047887     497        1  0.001126
192   -0.033419  0.047887     498        1  0.001126
190   -0.033419  0.047887     499        1  0.001126
499   -0.033419  0.047887     500        1  0.001126

[500 rows x 5 columns], topic_info=            Term           Freq          Total  Category  logprob  loglift
862     Ergebnis  126387.000000  126387.000000   Default  30.0000  30.0000
4683  aufwendung  149490.000000  149490.000000   Default  29.0000  29.0000
192        Seite  117502.000000  117502.000000   Default  28.0000  28.0000
360           nr  105288.000000  105288.000000   Default  27.0000  27.0000
206   ordentlich   96931.000000   96931.000000   Default  26.0000  26.0000
...          ...            ...            ...       ...      ...      ...
19         Stadt       0.000241   95622.048897  Topic500 -12.5202  -8.4054
23        darauf       0.000241     467.680357  Topic500 -12.5202  -3.0851
25      erklären       0.000241    2043.929139  Topic500 -12.5202  -4.5599
26         gemäß       0.000241   25670.289757  Topic500 -12.5202  -7.0904
28     mitteilen       0.000241     318.302904  Topic500 -12.5202  -2.7003

[20067 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
660      94  0.999801           --
4088     36  0.999954           AG
481       4  0.998807        Abbau
2268     26  0.998865   Abgrenzung
482      80  0.999731   Abrechnung
...     ...       ...          ...
3063      4  0.528485   übertragen
3063     16  0.471159   übertragen
2193     85  0.996320   überwachen
3581     75  0.998723  überörtlich
2195     37  0.997925       üblich

[1270 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 206, 300, 86, 12, 408, 155, 399, 29, 418, 315, 411, 415, 435, 81, 164, 178, 258, 88, 58, 361, 26, 190, 340, 181, 332, 171, 457, 373, 35, 30, 130, 444, 140, 366, 461, 290, 174, 195, 215, 98, 472, 75, 131, 216, 314, 44, 425, 317, 291, 289, 122, 441, 19, 450, 490, 231, 96, 213, 341, 210, 487, 266, 499, 200, 351, 22, 416, 99, 97, 310, 420, 485, 403, 331, 103, 322, 188, 493, 422, 273, 303, 60, 357, 474, 166, 112, 243, 488, 139, 471, 442, 497, 308, 383, 278, 110, 334, 72, 201, 102, 100, 25, 66, 389, 489, 382, 456, 397, 228, 67, 254, 480, 330, 40, 203, 335, 221, 282, 87, 196, 108, 438, 264, 376, 23, 367, 223, 144, 368, 265, 34, 316, 428, 182, 295, 429, 194, 251, 463, 462, 384, 47, 460, 323, 363, 105, 150, 304, 184, 249, 276, 90, 192, 104, 64, 197, 119, 65, 80, 250, 326, 146, 378, 21, 359, 437, 328, 311, 344, 360, 343, 313, 336, 312, 342, 345, 309, 362, 307, 306, 305, 337, 302, 301, 333, 348, 346, 347, 353, 350, 327, 354, 355, 349, 325, 329, 324, 338, 356, 321, 320, 358, 319, 318, 352, 339, 402, 364, 432, 465, 464, 459, 458, 455, 454, 453, 452, 451, 449, 448, 447, 446, 445, 443, 440, 439, 436, 434, 466, 467, 468, 483, 498, 496, 495, 494, 492, 491, 486, 484, 482, 469, 481, 479, 478, 477, 476, 475, 473, 470, 433, 431, 365, 430, 394, 393, 392, 391, 390, 388, 387, 386, 385, 381, 380, 379, 377, 375, 374, 372, 371, 370, 369, 395, 396, 398, 413, 427, 426, 424, 423, 421, 419, 417, 414, 412, 400, 410, 409, 407, 406, 405, 404, 298, 401, 299, 1, 297, 94, 116, 115, 114, 113, 111, 109, 107, 106, 101, 95, 93, 74, 92, 91, 89, 85, 84, 83, 82, 79, 78, 77, 117, 118, 120, 121, 149, 148, 147, 145, 143, 142, 141, 138, 137, 136, 135, 134, 133, 132, 129, 128, 127, 126, 125, 124, 123, 76, 73, 296, 16, 36, 33, 32, 31, 28, 27, 24, 20, 18, 17, 14

Notizen:
* 90 beinhaltet Radweg
* Verkehrsthemen sind schwer zu identifizieren
* 

#### 800 Themen

In [4]:
lda_tfidf_800 = models.ldamodel.LdaModel.load("data/lda_models/lda_model_tfidf_800")

In [ ]:
vis_tfidf_800 = pyLDAvis.gensim.prepare(lda_tfidf_500, corpus, id2word, mds='mmds')
vis_tfidf_800

### TDF

#### 500 Themen

In [3]:
from gensim import  models
lda_tdf_500 = models.ldamodel.LdaModel.load("data/lda_models/lda_model_tdf_500")
id2word_tdf_500 = corpora.Dictionary.load('data/lda_models/lda_model_tdf_500.id2word')
corpus_tdf_500 = corpora.MmCorpus("data/lda_models/tdidf_corpus")

In [4]:
vis_tdf_500 = pyLDAvis.gensim.prepare(lda_tdf_500, corpus_tdf_500, id2word_tdf_500, mds='mmds')
vis_tdf_500

C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.544707 -0.284241       1        1  5.590264
163    0.344105 -0.493156       2        1  5.338511
484    0.417559 -0.430189       3        1  3.570035
410    0.363770 -0.482489       4        1  3.527847
85     0.136864 -0.600781       5        1  2.826116
...         ...       ...     ...      ...       ...
320   -0.081629  0.070500     496        1  0.000735
93    -0.081629  0.070500     497        1  0.000735
324   -0.081629  0.070500     498        1  0.000735
92    -0.081629  0.070500     499        1  0.000735
499   -0.081629  0.070500     500        1  0.000735

[500 rows x 5 columns], topic_info=             Term           Freq          Total  Category  logprob  loglift
19          Stadt  114532.000000  114532.000000   Default  30.0000  30.0000
80          Kreis  122985.000000  122985.000000   Default  29.0000  29.0000
192         Seit

#### 800 Themen

In [6]:
lda_tdf_800 = models.ldamodel.LdaModel.load("data/lda_models/lda_model_")

FileNotFoundError: [Errno 2] No such file or directory: 'data/lda_models/lda_model_tdf_800'

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(1,
  '0.000*"begutachen" + 0.000*"stammdaen" + 0.000*"örtlichen_erhebungen" + '
  '0.000*"örtlichen_trägers" + 0.000*"Gewährleisteung" + 0.000*"Kontr" + '
  '0.000*"Nachbearbeitung" + 0.000*"ßerung" + 0.000*"Beanstandete" + '
  '0.000*"Kraft_gesetzt"'),
 (24,
  '0.000*"begutachen" + 0.000*"stammdaen" + 0.000*"örtlichen_erhebungen" + '
  '0.000*"örtlichen_trägers" + 0.000*"Gewährleisteung" + 0.000*"Kontr" + '
  '0.000*"Nachbearbeitung" + 0.000*"ßerung" + 0.000*"Beanstandete" + '
  '0.000*"Kraft_gesetzt"'),
 (10,
  '0.000*"dr_brud" + 0.000*"straßendat" + 0.000*"Beanstandete" + '
  '0.000*"Gewährleisteung" + 0.000*"Kontr" + 0.000*"Nachbearbeitung" + '
  '0.000*"begutachen" + 0.000*"stammdaen" + 0.000*"örtlichen_erhebungen" + '
  '0.000*"Pflichtgemäß"'),
 (20,
  '0.007*"bernd_leidig" + 0.003*"rm_sonja_schönherr" + '
  '0.002*"rm_christian_baldauf" + 0.001*"dr_günther_serfas" + '
  '0.000*"rm_daniel_kühner" + 0.000*"rm_dr_rainer" + '
  '0.000*"mitglieder_stellvertretung" + 0.000*"Ingrid_h

## Compute Model Perplexity and Coherence Score

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Visiualize Topic Model

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/opt/conda/lib/python3.10/site-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/opt/conda/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.395137 -0.252007       1        1  15.098063
29     0.122423 -0.445130       2        1   9.552139
28    -0.435777 -0.092291       3        1   8.964809
15     0.369316 -0.382388       4        1   7.966862
12     0.416934 -0.197640       5        1   7.563684
8      0.450991 -0.073457       6        1   6.609428
0      0.459551  0.174824       7        1   5.089331
17     0.198690 -0.320715       8        1   4.955507
6     -0.315073 -0.356347       9        1   4.815641
2     -0.171596 -0.392125      10        1   4.114136
22    -0.507736  0.074562      11        1   3.696108
5     -0.375334  0.251380      12        1   3.373255
21    -0.067586 -0.240166      13        1   3.014987
3     -0.048077 -0.485171      14        1   2.943910
13     0.159305  0.438184      15        1   2.146980
25    -0.110741  0.417627      16        1   2.130073
27    -0.230672  0.279147      17        1   2.051477
4     -0.229253  0.035024      18        1   1.646761
23     0.012082 -0.048559      19        1   1.283547
19     0.199372 -0.000237      20        1   1.165230
26     0.260666  0.198740      21        1   1.056249
9      0.075584  0.208163      22        1   0.282959
18    -0.036337  0.192884      23        1   0.275901
16     0.036156  0.157907      24        1   0.087982
7      0.021779  0.144381      25        1   0.037672
20     0.026768  0.141825      26        1   0.027108
11     0.030227  0.141008      27        1   0.025209
10     0.027825  0.143525      28        1   0.008416
1      0.027825  0.143525      29        1   0.008291
24     0.027825  0.143525      30        1   0.008284, topic_info=                 Term          Freq         Total Category  logprob  loglift
1094             Plan  42044.000000  42044.000000  Default  30.0000  30.0000
453            Straße  17368.000000  17368.000000  Default  29.0000  29.0000
2005          sonstig  26834.000000  26834.000000  Default  28.0000  28.0000
97    Stadtverwaltung  25760.000000  25760.000000  Default  27.0000  27.0000
19              Stadt  21915.000000  21915.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
24           erklären      0.000974    777.822921  Topic30 -11.8273  -4.1920
25              gemäß      0.000974  11363.648774  Topic30 -11.8273  -6.8737
26       insbesondere      0.000974   4418.421221  Topic30 -11.8273  -5.9291
27          mitteilen      0.000974    209.658408  Topic30 -11.8273  -2.8810
28         namentlich      0.000974     53.255727  Topic30 -11.8273  -1.5107

[1404 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
687       7  0.005747           --
687       8  0.993647           --
4145      5  0.997834           AG
182      17  0.999075       Absatz
699       1  0.202488      Abstand
...     ...       ...          ...
2222      1  0.999295   übernehmen
2228      7  0.998145  überschritt
3102      1  0.573128   übertragen
3102     14  0.425283   übertragen
3636     17  0.991112  überörtlich

[1358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 30, 29, 16, 13, 9, 1, 18, 7, 3, 23, 6, 22, 4, 14, 26, 28, 5, 24, 20, 27, 10, 19, 17, 8, 21, 12, 11, 2, 25])